In [201]:
# from pyspark.sql import SparkSession
# import sys
# from random import random
# from operator import add

# spark = SparkSession.builder.master("spark://10.1.174.144:7077").appName("PyCount Count").getOrCreate()

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row
spark = SparkSession \
.builder \
.appName("mongodbtest1") \
.master("spark://10.1.174.144:7077")\
.config("spark.mongodb.read.uri", "mongodb://abhi_1:60000/shardDB.asofashion_2") \
.config("spark.mongodb.write.uri", "mongodb://abhi_1:60000/shardDB.asofashion_clean") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.13:10.2.2') \
.getOrCreate()

In [217]:
# spark.stop()

In [203]:
spark

In [42]:
dataFrame = spark.read\
                 .format("mongodb")\
                 .option("connection.uri", "mongodb://abhi_1:60000/")\
                 .option("database", "shardDB")\
                 .option("collection", "asofashion_2")\
                 .load()

In [43]:
dataFrame.count()

140

In [109]:
dataFrame.take(1)

[Row(_id='660f3b689c0dd3afddadd33c', brand_name='Brave Soul', colour=nan, currency='USD', current_price=23.0, gender='male', previous_price=nan, productCode=119545541, productType='Product', product_id=203081399, rrp=42.0, title='Brave Soul cotton check shirt in off white & brown')]

In [101]:
from pyspark.sql import Row

def ser(x):
    x_dict = x.asDict()
    x_dict["title"] = x_dict["title"].lower()+ "_abcd"
    return Row(x_dict)

In [102]:
k = dataFrame.rdd.map(ser)

In [103]:
k.take(1)

[Row(_id='660f3b689c0dd3afddadd33d', brand_name='adidas Originals', colour=nan, currency='USD', current_price=54.72, gender='male', previous_price=99.48, productCode=114983404, productType='Product', product_id=201828775, rrp=nan, title="adidas originals 'sports resort' three stripe wide leg track pants in wonder white_abcd")]

In [110]:
k.count()

140

In [113]:
# spark.stop()

In [84]:
k.take(1)

[pyspark.sql.types.Row]

In [213]:
from pyspark.sql import Row

# def ser(x):
#     x_dict = x.asDict()
#     x_dict["title"] = x_dict["title"].lower()+ "_abcd"
#     print(x_dict["title"])

def ser(x):
    print(x)

In [216]:
stream_query = (spark.readStream
  .format("mongodb")
  .option("spark.mongodb.connection.uri", "mongodb://abhi_1:60000/")
  .option('spark.mongodb.database', "shardDB")
  .option('spark.mongodb.collection', "asofashion_2")
  .option("change.stream.publish.full.document.only","true")\
  .load()
  .writeStream
    .format("mongodb")
    .option("checkpointLocation", "./stream_meta/")
    .option("forceDeleteTempCheckpointLocation", "true")
    .option("connection.uri", "mongodb://abhi_1:60000/")
    .option("database", "shardDB")
    .option("collection", "asofashion_2")
    .outputMode("append")
)

y = stream_query.start()


24/04/10 20:16:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/10 20:16:32 WARN CaseInsensitiveStringMap: Converting duplicated key checkpointLocation into CaseInsensitiveStringMap.
24/04/10 20:16:32 WARN CaseInsensitiveStringMap: Converting duplicated key forcedeletetempcheckpointlocation into CaseInsensitiveStringMap.
24/04/10 20:16:32 WARN TaskSetManager: Lost task 0.0 in stage 4.0 (TID 16) (10.1.174.107 executor 0): java.lang.NoSuchMethodError: 'scala.collection.immutable.Seq org.apache.spark.sql.types.StructType.toAttributes()'
	at com.mongodb.spark.sql.connector.schema.InternalRowToRowFunction.<init>(InternalRowToRowFunction.java:46)
	at com.mongodb.spark.sql.connector.schema.RowToBsonDocumentConverter.createInternalRowToRowFunction(RowToBsonDocumentConverter.java:311)
	at com.mongodb.spark.sql.connector.schema.RowToBsonDocumentConverter.<init>(RowToBsonDocumentConverter.java:89)
	at com.mongod

In [210]:

# read_stream.select("title").collect()

# type(read_stream)

In [207]:
# read_stream.writeStream\
# .format("memory")\
# .option("forceDeleteTempCheckpointLocation","true")\
# .outputMode("append")\
# .queryName("abc")\
# .option("path","./check_path/")\
# .start()

In [208]:
# read_stream.collect()